In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [3]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import plot as pl
import process_ground_truth as pgt
import glob
import eval_boundary as ev
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci

In [4]:
ground_path = 'BSDS500/data/groundTruth/val/'
train_path = 'BSDS500/data/images/val/'
save_path = 'save/'
bound_prefix = 'images/'
eval_prefix = 'eval/'
extension = '.npz'

epsilon = 0.0001 #just to round down
number_clusters = 5

### Read Groundtruth

In [5]:
len_ground = len(ground_path)
ground_files = glob.glob(ground_path + '*.mat')
ground_filenames = [x[len_ground:-4] for x in ground_files]

groundtruths, images, names = [], [], []

#load images
for filename in ground_filenames:
    ground_file = ground_path + filename + '.mat'
    train_file = train_path + filename + '.jpg'
    
    #read groundtruth and image
    groundtruth = pgt.get_groundTruth(ground_file)
    image = img_as_float(io.imread(train_file))
    
    #append image and groundtruths
    groundtruths.append(groundtruth)
    images.append(image)
    names.append(filename + '.png')
    
print(len(groundtruths), len(images))

100 100


### Generate boundaries and save results

In [12]:
ini = 0
fim = len(ground_filenames)

methods = ['hslic', 'hsgb'] #['slic', 'sgb', 'egb']

for method in methods:
    last_shape = None
    blank_image = None
    index = 0
    
    starttime = time.time()
    for image, name in zip(images[ini:fim], names[ini:fim]):
        if(image.shape != last_shape):
            blank_image = np.zeros(image.shape,dtype=np.uint8) #create blank image to save
            last_shape = image.shape
        
        ##generate ultrametric map
        if method.find('h') >= 0:
            ultra_image, cluster_sizes = pipe.generate_ultrametric_image(image, blank_image, method, True)
        else:
            ultra_image = pipe.generate_boundaries(image, blank_image, method, black_color = True)
            cluster_sizes = 0
        
        #convert to grayscale
        gray_image = np.dot(ultra_image[...,:3], [0.299, 0.587, 0.114])
        
        #save image
        boundname = save_path + method + '/' + bound_prefix + name
        io.imsave(boundname, gray_image)
        
        #verbose mode
        index += 1
        if index % 10 == 0:
            print(index, cluster_sizes)

    endtime = time.time()
        
    #np.savez(boundname, borders=borders)
    print((endtime - starttime), 'seconds')

10 [130, 110, 90, 70]
20 [130, 110, 90, 70]
30 [130, 110, 90, 70]
40 [130, 110, 90, 70]
50 [130, 110, 90, 70]
60 [130, 110, 90, 70]
70 [130, 110, 90, 70]
80 [130, 110, 90, 70]
90 [130, 110, 90, 70]
100 [130, 110, 90, 70]
232.09186697006226 seconds
10 [40, 32, 24, 16, 8]
20 [40, 32, 24, 16, 8]
30 [40, 32, 24, 16, 8]
40 [40, 32, 24, 16, 8]
50 [40, 32, 24, 16, 8]
60 [40, 32, 24, 16, 8]
70 [40, 32, 24, 16, 8]
80 [40, 32, 24, 16, 8]
90 [40, 32, 24, 16, 8]
100 [40, 32, 24, 16, 8]
359.2053155899048 seconds
